In [ ]:
#| default_exp airsim.commands


# Drone Commands 
> Multicopter higher level commands for the Airsim simulator



In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

In [ ]:
#| export

from fastcore.utils import *
import UAV.airsim_python_client as airsim
from UAV.airsim_client import AirSimClient
import time
from UAV.utils.sim_linux import RunSim, is_process_running, find_and_terminate_process
import logging


In [ ]:
#| export
logging.basicConfig(format='%(asctime)-8s,%(msecs)-3d %(levelname)5s [%(filename)10s:%(lineno)3d] %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.DEBUG)  # Todo add this to params
logger = logging.getLogger(__name__)

In [ ]:
#| hide

from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
%%capture
from UAV.utils.display import *
from UAV.utils.sim_linux import *
from matplotlib import pyplot as plt

19:44:46,304 DEBUG [__init__.py:305] matplotlib data path: /home/jn/PycharmProjects/UAV/venv/lib/python3.10/site-packages/matplotlib/mpl-data
19:44:46,308 DEBUG [__init__.py:305] CONFIGDIR=/home/jn/.config/matplotlib
19:44:46,309 DEBUG [__init__.py:1479] interactive is False
19:44:46,309 DEBUG [__init__.py:1480] platform is linux
19:44:46,339 DEBUG [__init__.py:305] CACHEDIR=/home/jn/.cache/matplotlib
19:44:46,342 DEBUG [font_manager.py:1543] Using fontManager instance from /home/jn/.cache/matplotlib/fontlist-v330.json


In [ ]:
#| export
def start_sim():
    """Start the Airsim simuator if it is not already running"""
    sim_name = "AirSimNH"
    if not is_process_running(f"{sim_name}"):
        return RunSim("AirSimNH", settings="config/settings_high_res.json")
    return None   

In [ ]:
#|eval: false
show_doc(start_sim)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/airsim/commands.py#L22){target="_blank" style="float:right; font-size:smaller"}

### start_sim

>      start_sim ()

Start the Airsim simuator if it is not already running

In [ ]:
#| export
class DroneCommands():
    """Class Multirotor Client for the Airsim simulator with higher level procedures"""

    def __init__(self,
                 takeoff_z:int = -5): # takeoff Height
        self._z = takeoff_z
        self._stop = False

    def arm(self):
            """Run the drone on a path in the Airsim simulator.
            Creates a client and connects to the simulator."""
            self._client = AirSimClient()
    
            self._client.enableApiControl(True)
        
            logger.info("Arming the drone...")
            self._client.armDisarm(True)        


    def disarm(self):
        """Disarm the drone and disconnect from the simulator"""
        print("disarming...")
        self._client.armDisarm(False)
        self._client.enableApiControl(False)
        
        
    def takeoff(self):
        """Takeoff to the takeoff height"""
        state = self._client.getMultirotorState()
        if state.landed_state == airsim.LandedState.Landed:
            print("taking off...")
            self._client.takeoffAsync().join()
        else:
            self._client.hoverAsync().join()
    
        time.sleep(1)
    
        state = self._client.getMultirotorState()
        if state.landed_state == airsim.LandedState.Landed:
            print("take off failed...")
            sys.exit(1)
    
        # AirSim uses NED coordinates so negative axis is up.
        # _z of -5 is 5 meters above the original launch point.
        # _z = -50
        print("make sure we are hovering at {} meters...".format(-self._z))
        self._client.moveToZAsync(self._z, 5).join()

    
    def do_NH_path(self):
        """Fly on a path in the Airsim simulator"""
        print("flying on path...")
        print("""This script is designed to fly on the streets of the Neighborhood environment
            and assumes the unreal position of the drone is [160, -1500, 120].""")
        result = self._client.moveOnPathAsync([airsim.Vector3r(125,0,self._z),
                                        airsim.Vector3r(125,-130,self._z),
                                        airsim.Vector3r(0,-130,self._z),
                                        airsim.Vector3r(0,0,self._z)],
                                12, 120,
                                airsim.DrivetrainType.ForwardOnly, airsim.YawMode(False,0), 20, 1).join()
    
    def rth(self):
        print("returning home...")
        # drone will over-shoot so we bring it back to the start point before landing.
        # _client.moveToPositionAsync(0,0,_z,1).join()
        self._client.goHomeAsync().join()
    
    def land(self):
        print("landing...")
        self._client.landAsync().join()
    

    def do_tasklist(self):
        """Run a list of tasks in order"""
        tasks = [self.arm, self.takeoff, self.do_NH_path, self.rth, self.land, self.disarm]
        for task in tasks:
            if self._stop:
                break
            task()
                
    def stop(self):
        """ stop the client by cancelling the last task and exiting the do_tasklist loop """
        self._stop = True
        try:
            self._client.cancelLastTask()
        except Exception as e:
            print (e)


In [ ]:
#|eval: false
show_doc(DroneCommands.arm)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/airsim/commands.py#L38){target="_blank" style="float:right; font-size:smaller"}

### DroneCommands.arm

>      DroneCommands.arm ()

Run the drone on a path in the Airsim simulator.
Creates a client and connects to the simulator.

In [ ]:
#|eval: false
show_doc(DroneCommands.takeoff)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/airsim/commands.py#L56){target="_blank" style="float:right; font-size:smaller"}

### DroneCommands.takeoff

>      DroneCommands.takeoff ()

Takeoff to the takeoff height

In [ ]:
#|eval: false
show_doc(DroneCommands.do_NH_path)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/airsim/commands.py#L79){target="_blank" style="float:right; font-size:smaller"}

### DroneCommands.do_NH_path

>      DroneCommands.do_NH_path ()

Fly on a path in the Airsim simulator

In [ ]:
#|eval: false
show_doc(DroneCommands.rth)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/airsim/commands.py#L91){target="_blank" style="float:right; font-size:smaller"}

### DroneCommands.rth

>      DroneCommands.rth ()

In [ ]:
#|eval: false
show_doc(DroneCommands.land)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/airsim/commands.py#L97){target="_blank" style="float:right; font-size:smaller"}

### DroneCommands.land

>      DroneCommands.land ()

In [ ]:
#|eval: false
show_doc(DroneCommands.disarm)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/airsim/commands.py#L49){target="_blank" style="float:right; font-size:smaller"}

### DroneCommands.disarm

>      DroneCommands.disarm ()

Disarm the drone and disconnect from the simulator

In [ ]:
#|eval: false
show_doc(DroneCommands.do_tasklist)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/airsim/commands.py#L102){target="_blank" style="float:right; font-size:smaller"}

### DroneCommands.do_tasklist

>      DroneCommands.do_tasklist ()

Run a list of tasks in order

In [ ]:
#|eval: false
show_doc(DroneCommands.stop)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/airsim/commands.py#L110){target="_blank" style="float:right; font-size:smaller"}

### DroneCommands.stop

>      DroneCommands.stop ()

stop the client by cancelling the last task and exiting the do_tasklist loop

### An end to end example

The drone cammands are run in a separate process with the video loop running in the main process.

In [ ]:
""" """
import threading
from UAV.airsim.commands import DroneCommands
from UAV.airsim.client import  AirSimClient
from UAV.utils.sim_linux import RunSim
from UAV.utils.display import puttext, VideoWriter, ScrollingLog
from imutils import resize
import cv2

rs = RunSim("AirSimNH", settings="config/settings_high_res.json")

asc = AirSimClient()
cmd = DroneCommands()
t = threading.Thread(target=cmd.do_tasklist, daemon=True)
t.start()

framecounter = 1
cam_num = 0
cams = ["high_res", "front_center", "front_right", "front_left", "bottom_center", "back_center"]
# with VideoWriter("images/airsim_test.mp4", 5.0) as video:
with VideoWriter("images/airsim_nav_test.mp4", 5.0) as video:
    log = ScrollingLog(position=(50,200), font_scale=1 , bg_color=(0,0,0))
    while(True):
        framecounter += 1
        state = asc.getMultirotorState()
        pos = state.kinematics_estimated.position
        img = asc.get_image(cams[cam_num], rgb2bgr=False)
        puttext(img, f"Frame: {framecounter} Pos: {pos.x_val:.2f}, {pos.y_val:.2f}, {pos.z_val:.2f}")
        log = ScrollingLog(font_scale=1 , bg_color=(0,0,0))
        img = resize(img, width=500)
        # log.update(f"Frame: {framecounter} Pos: {pos.x_val:.2f}, {pos.y_val:.2f}, {pos.z_val:.2f}")
        log.draw(img)
        cv2.imshow("Camera", img)
        
        # video.add(img_bgr)
        k = cv2.waitKey(10)
        if k == ord('q') or k == ord('Q'):
            log.update("cancelLastTask")
            asc.cancelLastTask()
            # print(f"Landed state:  {state.landed_state}")
            if state.landed_state == 0:
                log.update("Landed state = 0,  so quiting")
                break  
        
        if k == ord('c') or k == ord('C'):
            cam_num += 1
            if cam_num >= len(cams):
                cam_num = 0
            log.update(f"Camera: {cams[cam_num]}")
            
        if k == 27:
            cmd.stop()
            time.sleep(1)
            break
            
        # if framecounter > 50:
        #     break

cmd.disarm()
t.join(timeout=5)
cv2.destroyAllWindows()
rs.exit()

video.show(width=500)

19:58:02,575 DEBUG [__init__.py:305] matplotlib data path: /home/jn/PycharmProjects/UAV/venv/lib/python3.10/site-packages/matplotlib/mpl-data
19:58:02,579 DEBUG [__init__.py:305] CONFIGDIR=/home/jn/.config/matplotlib
19:58:02,580 DEBUG [__init__.py:1479] interactive is False
19:58:02,581 DEBUG [__init__.py:1480] platform is linux
19:58:02,610 DEBUG [__init__.py:305] CACHEDIR=/home/jn/.cache/matplotlib
19:58:02,613 DEBUG [font_manager.py:1543] Using fontManager instance from /home/jn/.cache/matplotlib/fontlist-v330.json


Starting Airsim  ['/home/jn/Airsim/AirSimNH/LinuxNoEditor/AirSimNH/Binaries/Linux/AirSimNH', '-ResX=800', '-ResY=600', '-windowed', '-settings=/home/jn/PycharmProjects/UAV/UAV/config/settings_high_res.json']
Started Airsim AirSimNH


19:58:05,794  INFO [commands.py: 45] Arming the drone...


Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)

Writing video to /home/jn/PycharmProjects/UAV/nbs/api/images/airsim_nav_test.mp4 at 5.0 fps.
Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)

taking off...
make sure we are hovering at 5 meters...
flying on path...
This script is designed to fly on the streets of the Neighborhood environment
            and assumes the unreal position of the drone is [160, -1500, 120].
returning home...
landing...
disarming...
disarming...
Airsim exited with rc = 143
Video: /home/jn/PycharmProjects/UAV/nbs/api/images/airsim_nav_test.mp4


OSError: MoviePy error: the file images/airsim_nav_test.mp4 could not be found!
Please check that you entered the correct path.

In [ ]:
# rs.exit()

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()